In [1]:
import numpy as np

Supongamos que tenemos una matriz NumPy que representa algunos datos de un experimento:

In [44]:
temperature_0 = np.random.random(1024)

In [45]:
temperature_from_station_20=np.random.random(1024)

In [19]:
np.info(temperature_0)

class:  ndarray
shape:  (1024,)
strides:  (8,)
itemsize:  8
aligned:  True
contiguous:  True
fortran:  True
data pointer: 0x55dc3e391350
byteorder:  little
byteswap:  False
type: float64


Imaginemos también que estos puntos de datos se registraron desde una estación meteorológica que muestreó la temperatura, por ejemplo, cada 10 segundos. 

In [14]:
dt = 10.0


In [ ]:
La adquisición de datos comenzó en un momento particular,
que también tendremos que registrar. 

In [15]:
start_time = 1375204299 

In [ ]:
Y, por supuesto, tenemos que saber que los datos provienen
de la Estación Meteorológica 15:


In [16]:
station = 15

In [23]:
np.savez('weather.npz', data=temperature_0, start_time=start_time, station=station)

Podemos recuperar los valores del archivo con np.load:

In [24]:
out = np.load('weather.npz')

imprimimos "out"

In [25]:
out['data']

array([0.0774394 , 0.39690988, 0.65772249, ..., 0.07119171, 0.80505879,
       0.23065056])

NpzFile (fid)

Un objeto tipo diccionario con carga lenta de archivos en el archivo comprimido
archivo proporcionado en la construcción.

`NpzFile` se usa para cargar archivos en el archivo de datos NumPy` `.npz``
formato. Se supone que los archivos en el archivo tienen una extensión `` .npy``,
otros archivos son ignorados

Las matrices y las cadenas de archivos se cargan perezosamente en cualquiera
obtener acceso a elementos usando `` obj ['clave'] `` o búsqueda de atributos usando
`` obj.f.key ''. Una lista de todos los archivos (sin extensiones `` .npy``) puede
se obtendrá con `` obj.files '' y el objeto ZipFile en sí mismo usando
`` obj.zip ''.

Atributos
----------
archivos: lista de str
    Lista de todos los archivos en el archivo con una extensión `` .npy``.
zip: instancia de ZipFile
    El objeto ZipFile se inicializó con el archivo comprimido.
f: instancia de BagObj
    Un objeto en el que el atributo se puede realizar como alternativa
    para obtener acceso al elemento en la propia instancia de `NpzFile`.
allow_pickle: bool, opcional
    Permitir cargar datos encurtidos. Valor predeterminado: falso

    .. versionchanged :: 1.16.3
        Falso predeterminado en respuesta a CVE-2019-6446.

pickle_kwargs: dict, opcional
    Argumentos de palabras clave adicionales para pasar a pickle.load.
    Estos solo son útiles al cargar matrices de objetos guardadas en
    Python 2 cuando se usa Python 3.

Parámetros
----------
fid: archivo o str
    El archivo comprimido para abrir. Este es un objeto similar a un archivo
    o una cadena que contiene la ruta al archivo.
own_fid: bool, opcional
    Si NpzFile debería cerrar el identificador de archivo.
    Requiere que `fid` sea un objeto similar a un archivo.

Ejemplos
--------
>>> desde la importación de archivos temporales TemporaryFile
>>> archivo de salida = TemporaryFile ()
>>> x = np. naranja (10)
>>> y = np.sin (x)
>>> np.savez (outfile, x = x, y = y)
>>> _ = outfile.seek (0)

>>> npz = np.load (archivo de salida)
>>> isinstance (npz, np.lib.io.NpzFile)
Cierto
>>> ordenado (npz.files)
['x', 'y']
>>> npz ['x'] # getitem acceso
matriz ([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
>>> npz.fx # búsqueda de atributos
matriz ([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [26]:
out['start_time']

array(1375204299)

In [27]:
np.info(out['start_time'])

class:  ndarray
shape:  ()
strides:  ()
itemsize:  8
aligned:  True
contiguous:  True
fortran:  True
data pointer: 0x55dc3e355930
byteorder:  little
byteswap:  False
type: int64


In [28]:
out['station']

array(15)

In [29]:
np.info(out['station'])

class:  ndarray
shape:  ()
strides:  ()
itemsize:  8
aligned:  True
contiguous:  True
fortran:  True
data pointer: 0x55dc3e412de0
byteorder:  little
byteswap:  False
type: int64


Hasta aquí todo bien.
Pero, ¿qué pasa si tenemos más de una cantidad por estación? 

¿Digamos que también hay datos de velocidad del viento para registrar?



In [30]:
wind = np.random.random(2048)

In [31]:
dt_wind = 5.0   # Wind sampled every 5 secondsÇ

In [32]:
import h5py

In [43]:
f = h5py.File('weather.hdf5')

<ipython-input-43-bc60a7513f3d>:1: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  f = h5py.File('weather.hdf5')


In [34]:
 f['/15/temperature'] = temperature_0

In [35]:
f['/15/temperature'].attrs['dt'] = 10.0

In [36]:
f['/15/temperature'].attrs['start_time'] = 1375204299

In [37]:
f['/15/wind'] = wind

In [40]:
f['/15/wind'].attrs['dt'] = 5.0

In [46]:
f['/20/temperature'] = temperature_from_station_20

In [ ]:
Este ejemplo ilustra dos de las "características asesinas" de HDF5:
organización en grupos y atributos jerárquicos. 
Grupos, como carpetasen un sistema de archivos, le permite almacenar conjuntos de datos relacionados juntos. 
En este caso, las mediciones de temperatura y
viento de la misma estación meteorológica se almacenan juntas en grupos llamados "/ 15", "/ 20", etc. 
Los atributos le permiten adjuntar metadatos descriptivos directamente a los datos que describe .
Entonces, si le entrega este archivo a un colega, 
ella puede descubrir fácilmente la información necesaria para dar sentido a los datos:

In [47]:
dataset = f['/15/temperature']

In [49]:
for key, value in dataset.attrs.items():
    print ('%s: %s' % (key, value))

dt: 10.0
start_time: 1375204299
